In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/wujindou/cks23.git

fatal: destination path 'cks23' already exists and is not an empty directory.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !ls -la /content/drive/MyDrive/bart_seq2seq_task9

ls: cannot access '/content/drive/MyDrive/bart_seq2seq_task9': No such file or directory


In [ ]:
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import sys
def read_excel(filename):
  d = pd.read_excel(filename,sheet_name=[0])
  query = None
  text = None
  entities = []
  dataset = []
  all_count =0
  all_found = 0
  for data in d[0].values:
    if str(data[0])!='nan':
      if query:
        all_count+=len(entities)
        not_found_entities = []
        for entity in entities:
          if entity not  in text:
            not_found_entities.append(entity)
        dataset.append({'query':query,'text':text.strip(),'entities':';'.join(entities)})
        query = None
        text = None
        entities= []
      query = data[0]
      text = data[1]
    if str(data[2])!='nan':
      entities.append(data[2])
  if query:
    dataset.append({'query':query,'text':text.strip(),'entities':';'.join(entities)})
  return dataset


dataset = read_excel('./cks23/train_ccks.xls')
# print(len(dataset))

In [ ]:
# import json
# references = json.load(open('data_reference.json','r',encoding='utf-8'))
# for d,ref in zip(dataset,references):
#   d['reference'] = ref['reference']
#   if d['query']!=ref['query']:
#     print(d)
#     print(ref)


In [ ]:
print(dataset[0])

{'query': '湖北九有投资股份有限公司', 'text': '公司归属于造纸及纸制品行业,是一家以新闻纸生产为主业的制浆造纸企业.', 'entities': '造纸;纸制品;新闻纸', 'reference': '制浆造纸化学品;机制纸及纸板厂;纸机和造纸设备;造纸设备及配件;造纸材料;造纸化学品;机制纸浆;纸浆模塑制品;造纸厂;造纸设备;装饰原纸的研发;造纸色浆;销售纸张和印刷器材;试纸产品的研发;纸板的研发;造纸网;纸板生产设备;造纸黑液资源化利用;纸品加工;造纸酶'}


In [ ]:
import numpy as np
import random
import torch
from transformers import set_seed
set_seed(42)
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True
train_data = dataset[:600]+dataset[650:]
dev_data = dataset[600:650]
print(len(train_data))
print(len(dev_data))

650
50


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import datasets
# train_data = pd.DataFrame(converted_train_data)
train_dataset =datasets.Dataset.from_pandas(pd.DataFrame(train_data))
dev_dataset = datasets.Dataset.from_pandas(pd.DataFrame(dev_data))

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
checkpoint='fnlp/bart-large-chinese'
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint,use_fast=False)
def preprocess_function(example):
  model_inputs = tokenizer(example['text']+'[SEP]'+example['reference'], truncation=True, padding="max_length", max_length=256)
  labels = tokenizer(text_target=example["entities"], max_length=128, truncation=True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

ds={'train':train_dataset,'validation':dev_dataset}
for split in ds:
  ds[split] = ds[split].map(preprocess_function)

Map:   0%|          | 0/650 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
!pip install rouge_score rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import evaluate
from rouge import Rouge
rouge_score = Rouge()
# rouge = evaluate.load("rouge")

"""Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the ROUGE metric:"""

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print(decoded_preds[0]+'\t'+decoded_labels[0])
    decoded_labels=['no' if t.strip()=='' else t for t in decoded_labels]
    decoded_preds=['no' if t.strip()=='' else t for t in decoded_preds]

    scores = rouge_score.get_scores(decoded_preds,decoded_labels,avg=True)
    for key in scores:
      scores[key] = scores[key]['f']*100
    result = scores
    #result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

def get_model():
    # get_model is used for the model_init argument for trainer. This ensures reproducibility. Otherwise, weights from classification head are randomly initialized.
    # see https://discuss.huggingface.co/t/fixing-the-random-seed-in-the-trainer-does-not-produce-the-same-results-across-runs/3442
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    return model


In [ ]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !export CUDA_VISIBLE_DEVICES=1

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bart_seq2seq_task9",
    evaluation_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to='tensorboard',
    load_best_model_at_end=True,
    save_strategy='steps',
    eval_steps=20,
    save_steps=20,
    seed=42

)

trainer = Seq2SeqTrainer(
    model_init=get_model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Gen Len
0,No log,0.366372,73.991300,64.145900,72.811400,17.320000
2,No log,0.327724,76.580800,66.975600,75.340900,17.500000
2,No log,0.333880,77.280000,66.907600,76.023800,17.560000


轴 流 风 机 的 研 究 、 开 发 、 生 产 、 销 售 和	轴 流 、 混 流 、 离 心 风 机 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 风 冷 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 水 冷 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 速 冻 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务
高 新 技 术 产 品 的 研 究 、 开 发 、 生 产 、 销	轴 流 、 混 流 、 离 心 风 机 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 风 冷 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 水 冷 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 速 冻 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务
轴 流 风 机 的 研 究 、 开 发 、 生 产 、 销 售 和	轴 流 、 混 流 、 离 心 风 机 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 风 冷 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 水 冷 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务 ; 速 冻 装 备 的 研 究 、 开 发 、 生 产 、 销 售 和 技 术 服 务


TrainOutput(global_step=60, training_loss=0.5190938949584961, metrics={'train_runtime': 230.6572, 'train_samples_per_second': 8.454, 'train_steps_per_second': 0.26, 'total_flos': 1033708895207424.0, 'train_loss': 0.5190938949584961, 'epoch': 2.93})

In [ ]:
trainer.save_model('./bart_seq2seq_task9')

In [ ]:
from transformers import AutoTokenizer
# checkpoint ='t5-large'
# folder='/content/drive/MyDrive/bart_seq2seq_task9'
folder = './bart_seq2seq_task9'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(folder)

In [ ]:
import torch
device = torch.device('cuda')
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(51271, 1024, padding_idx=0)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(51271, 1024, padding_idx=0)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [ ]:
best_precision = 0
best_beam = 0
for beamsize in [1]:
    preds = []
    def inference(model,inputs,max_s_length=128,max_target_length=384):
        padding=True
        model_inputs = tokenizer(inputs,max_length=max_s_length, padding=padding, truncation=True,return_tensors='pt').input_ids
        outputs= model.generate(model_inputs.to(device),max_length=max_target_length)
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)

    def inference_with_file(dataset,batch_size=1):
        import json
        results =[]
        batch_data = []
        for idx,data in enumerate(dataset):
          batch_data.append(data['text']+'[SEP]'+data['reference'])
          if len(batch_data)==batch_size:
              # print(batch_data)
              # sys.exit(1)
              batch_preds = inference(model,batch_data)
              # print(batch_preds)
              batch_data = []
              results.extend(batch_preds)
        if len(batch_data)>0:
          batch_preds = inference(model,batch_data)
          results.extend(batch_preds)
        print(len(results))
        return results
    results = inference_with_file(dev_data)
    precision = 0
    found = 0
    recall = 0
    right = 0
    for d,pred in zip(dev_data,results):
      gold_entities = d['entities'].split(';')
      pred_entity = pred.replace(' ','').split(';')
      recall += len(gold_entities)
      for p in pred_entity:
        if p in gold_entities:
          right+=1
        found+=1
      print(d['text']+'\t pred='+pred.replace(' ','')+'\t entity='+d['entities'])
    precision = float(right)/found
    recall = float(right)/recall
    print('f1 = '+str(2*precision*recall/(precision+recall))+'\t p='+str(precision)+'\t r='+str(recall))
    if precision >best_precision :
        best_precision = precision
        best_beam = beamsize
        print('best_precsion = '+str(best_precision)+'\t'+str(beamsize))


50
各类轴流风机、混流风机、离心风机和各类风冷、水冷、速冻装备等高新技术产品的研究、开发、生产、销售和技术服务。	 pred=高新技术产品的研究、开发、生产、销售和技术服务	 entity=轴流、混流、离心风机的研究、开发、生产、销售和技术服务;风冷装备的研究、开发、生产、销售和技术服务;水冷装备的研究、开发、生产、销售和技术服务;速冻装备的研究、开发、生产、销售和技术服务
原煤、焦炭、精煤、中煤、煤气以及焦油、 轻苯等煤化工产品的生产和销售等	 pred=原煤、焦炭、精煤、中煤、煤气以及焦油、轻苯等煤化工产品的生产和销售	 entity=煤化工产品的生产和销售
新材料与制品及其生产工艺装备的研究、开发、设计、生产和销售。	 pred=新材料与制品及其生产工艺装备的研究、开发、设计、生产和销售	 entity=新材料的研究、开发、设计、生产和销售;生产工艺装备的研究、开发、设计、生产和销售
公司主要从事稀土永磁材料和新型磁性材料及其应用产品的研究开发、生产和销售	 pred=稀土永磁材料的研究开发、生产和销售	 entity=稀土永磁材料及其应用产品的研究开发、生产和销售;新型磁性材料及其应用产品的研究开发、生产和销售
高档精毛纺呢绒面料的开发、生产、销售为主要业务。	 pred=精毛纺呢绒面料的开发、生产、销售	 entity=高档精毛纺呢绒面料的开发、生产、销售
集番茄种植、生产、加工、贸易、科研开发为一体的农业产业化国家重点龙头企业.公司主要产品有番茄原料酱、番茄制品及蔬菜罐头制品等	 pred=番茄原料酱;番茄制品;蔬菜罐头制品	 entity=番茄种植、生产、加工、贸易、科研开发;番茄原料酱、番茄制品的生产销售;蔬菜罐头制品的生产、销售
各类塑料制品、粘胶制品、各类包装及印刷复合制品和塑料加工设备、模具的制造、加工及工程设计安装。	 pred=塑料制品的制造、加工及工程设计安装	 entity=塑料加工设备的制造、加工;塑料模具、制品的制造、加工及工程设计安装;粘胶制品的制造、加工及工程设计安装;印刷复合制品的制造、加工及工程设计安装;各类包装的制造、加工及工程设计安装
市政共用事业、环卫	 pred=市政共用事业;环卫	 entity=市政共用事业;环卫
生产、销售差别异型涤纶长丝、锦纶长丝等化纤产品	 pred=化纤产品的生产、销售	 entity=

In [ ]:
names = []
test_data = []
with open('./cks23/test_file.txt','r',encoding='utf-8') as lines:
  for line in lines:
    data = line.strip().split('\t')
    names.append(data[0])
    test_data.append({'query':data[0],'text':data[1]})

In [ ]:
results = inference_with_file(test_data)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in inference_with_file:16                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'reference'

In [ ]:
import json
writer = open('ner_bart_large_3_06262.txt','a+',encoding='utf-8')
for json_data,ner_result in zip(test_data,results):
  json_data['entities'] = ner_result
  writer.write(json.dumps(json_data,ensure_ascii=False)+'\n')
writer.close()